In [1]:
pip install fredapi


Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import numpy as np
import os
import re
from fredapi import Fred

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [20]:
pd.reset_option("all")

/var/folders/l4/rqys14ws2955nprxbdsj722r0000gn/T/ipykernel_79682/3365680546.py:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option("all")
/var/folders/l4/rqys14ws2955nprxbdsj722r0000gn/T/ipykernel_79682/3365680546.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option("all")


# I - Getting the data 

For the US, we use the FRED database, available on the St Louis website, which contains a wide range of macroeconomic variables with monthly frequency. We will extract data using the FRED API and then store everything in a dataframe.

### 1) Extracting data with the API

In [5]:
api_key = '615add3bd441c0ed42dec47e952d69e2'
fred = Fred(api_key=api_key)

After many iterations, we get the following variables, which take some values between 2003-01-02 and 2025-08-01. For each of these variables, we tried to get the highest frequency of observations proposed by FRED database:
- for most financial markets data, we have daily observations
- for most macroeconomic variables, we have monthly observations
- for most fiscal variables, we have quarterly observations

Here is a list of the variables we extracted. Unfortunately, some important variables such as gold or SP500 were missing, so we will have to get these datas somewhere else.

In [6]:


variables_to_keep = [#production and revenue variables
    'RPI',  #Real personal income; monitored by FED for interest rates decisions
                   'INDPRO', #Industrial production index; leading indicator of economic health, monitored by FED 
                   #job market variables
                   'UNRATE', #unemployment rate
                   'UEMP5TO14', #unemployment for people unemployed between 5 and 14 weeks = short term unemployment
                   'UEMP27OV', #unemployment for people unemployed for 27 weeks and over = long term unemployment
                    'PAYEMS', #total non farm payroll, leading indicator of employment
                    'USGOOD',   #below is the breakdown of employment figures for various economic sectors
                    'CES1021000001',
                    'USCONS',
                    'MANEMP',
                    'DMANEMP',
                    'NDMANEMP',
                    'SRVPRD',
                    'USTPU',
                    'USWTRADE',
                    'USTRADE',
                    'USFIRE',
                    'USGOVT',
                    #consumption and housing
                    'HOUST', #housing variable, sensitive to interest rates and economic anticipations
                    'PERMIT', #same but for building permits
                    'UMCSENT', #university of michigan consumer sentiment index, important for consumption anticipations
                    #money and credit variables
                    'M1SL', #monetary base, important for interest rates and inflation anticipations
                    'M2SL', 
                    'M2REAL', #real monetary base
                    'TOTRESNS', #total reserves of banks
                    'NONBORRES', #non borrowable reserves of banks
                    'BUSLOANS', #business loans, important for growth anticipations
                    'REALLN', #real loans, related to economic activity
                    'NONREVSL', #non-revolving consumer credit, related to consumption
                    'DTCOLNVHFNM', #total nonfinancial corporate business liabilities, related to economic activity
                    'DTCTHFNM', #total household liabilities, related to consumption
                    'INVEST', #gross private domestic investment, related to economic activity
                    #price indexes
                    'CPIAUCSL', #principal measure for inflation monitored by FED
                    'CPIAUCSL', 'CPIAPPSL', 'CPITRNSL','CPIMEDSL', 'CUSR0000SAC', 'CUUR0000SAD', 'CUSR0000SAS', 'CPIULFSL', 'CUUR0000SA0L2', 'CUSR0000SA0L5', #breakdown of the index for various sectors
                    'PCEPI', #inflation target for FED
                    'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA', #spendings for services, durable and nondurable goods
                    #fiscal variables
                    'FGEXPND', #quarterly federal government expenditures
                    'FGRECPT', #quarterly federal government receipts
                    'FGDEF',   #quarterly federal government budget deficit
                    'FGCE', # quarterly federal government spending
                    'MTSDS133FMS', # monthly federal surplus or deficit
                    'FYGFDPUN', #federal debt held by the public, quarterly 
                     'GFDEGDQ188S', #debt to gdp ratio, quarterly 
                     'A091RC1Q027SBEA', #net interest payment, quarterly 
                     'GFDEBTN', #total debt outstanding, monthly 
                     #yield variables
                     'DGS1MO', 'DGS3MO', 'DGS6MO', #yields on treasury bills with maturities < 1y
                    'DGS1', 'DGS2', 'DGS3','DGS5','DGS7', 'DGS10', 'DGS20', 'DGS30', #yields of government treasuries for maturies >=1y
                    #'GS1M','TB3MS','TB6MS', #yield on commercial paper
                    #'GS1', 'GS2', 'GS3','GS5','GS7', 'GS10', 'GS20', 'GS30', #yields of government treasuries for maturies >=1y  
                    #yields of other "safe haven"
                    'IRLTLT01DEM156N', #10y german bund yield
                    'IRLTLT01JPM156N', #10y japanese gov yield
                    'IRLTLT01GBM156N', #10y uk gilt yield
                    'IRLTLT01CAM156N', #10y canadian gov yield
                    'IRLTLT01AUM156N', #10y australian gov yield
                    'IRLTLT01FRM156N', #10y french gov yield
                    #stock markets
                    'NASDAQCOM', #nasdaq
                    #USD FX rates and US corporate bonds variables
                    'AAA', 'BAA', #yields of corporate bonds with notations AAA and BAA
                    'DEXUSEU', #usd to euro exchange rate
                    'DEXJPUS', #usd to yen exchange rate
                    'DEXUSUK', #usd to gbp exchange rate
                    'DEXCAUS', #usd to cad exchange rate
                    'DEXUSAL', #usd to aud exchange rate
                    'DEXCHUS', #usd to yuan exchange rate
                    #other financial variables measuring risk on / risk off 
                    'VIXCLS', #vix index, measure of market volatility
                    'NFCI', #national financial conditions index
                    'DCOILWTICO', #crude oil price, important for inflation anticipation
                     #interest rates variables (traditional monetary policy)
                    'FEDFUNDS', #federal funds rate, short term interest rates
                    'WRESBAL', #measure of liquidity in the banking system
                    'CURRCIR', #currency in circulation
                    'WTREGEN', #government deposits 
                    #QE and QT variables 
                    'BOGMBASE', #monetary base
                    'WSHOSHO', #total assets of FED = size of balance sheet
                    'WSHOTSL', #total treasuries held by FED
                    'WSHOBL', #total longer duration trasuries held by FED (for long end of yield curve)
                    #anticipations of inflation by markets
                    'T5YIE', #5y forward inflation expectation
                    'T10YIE', #10y forward inflation expectation

                    ]




start_date = '2003-01-02' #we don't do a start date because lots of data are missing before that 
end_date   = '2025-08-01'


In [7]:
data = {}
for code_variable in variables_to_keep:
    print(code_variable)
    try:
        series = fred.get_series(code_variable, observation_start=start_date, observation_end=end_date)
        data[code_variable] = series
    except ValueError:
        print(f"Could not retrieve data for {code_variable}")
        continue

RPI
INDPRO
UNRATE
UEMP5TO14
UEMP27OV
PAYEMS
USGOOD
CES1021000001
USCONS
MANEMP
DMANEMP
NDMANEMP
SRVPRD
USTPU
USWTRADE
USTRADE
USFIRE
USGOVT
HOUST
PERMIT
UMCSENT
M1SL
M2SL
M2REAL
TOTRESNS
NONBORRES
BUSLOANS
REALLN
NONREVSL
DTCOLNVHFNM
DTCTHFNM
INVEST
CPIAUCSL
CPIAUCSL
CPIAPPSL
CPITRNSL
CPIMEDSL
CUSR0000SAC
CUUR0000SAD
CUSR0000SAS
CPIULFSL
CUUR0000SA0L2
CUSR0000SA0L5
PCEPI
DDURRG3M086SBEA
DNDGRG3M086SBEA
DSERRG3M086SBEA
FGEXPND
FGRECPT
FGDEF
FGCE
MTSDS133FMS
FYGFDPUN
GFDEGDQ188S
A091RC1Q027SBEA
GFDEBTN
DGS1MO
DGS3MO
DGS6MO
DGS1
DGS2
DGS3
DGS5
DGS7
DGS10
DGS20
DGS30
IRLTLT01DEM156N
IRLTLT01JPM156N
IRLTLT01GBM156N
IRLTLT01CAM156N
IRLTLT01AUM156N
IRLTLT01FRM156N
NASDAQCOM
AAA
BAA
DEXUSEU
DEXJPUS
DEXUSUK
DEXCAUS
DEXUSAL
DEXCHUS
VIXCLS
NFCI
DCOILWTICO
FEDFUNDS
WRESBAL
CURRCIR
WTREGEN
BOGMBASE
WSHOSHO
WSHOTSL
WSHOBL
T5YIE
T10YIE


To account for missing data on weekly, monthly, quarterly time series, we'll use forward fill. We will also use backward fill after that to fill the first values missing in january 2003. This should not introduce any forward bias as all time series have a start date before the 8th of January 2003.

In [8]:
df = pd.concat(data, axis=1)
df.columns = data.keys()
df.to_csv('data_fred.csv',index = True)

In [27]:
data_path = os.path.join('data', 'US', 'data_fred.csv')
df = pd.read_csv(data_path, index_col=0, parse_dates=True)
df.index = pd.to_datetime(df.index, utc=True)
df = df.ffill()
df = df.bfill()
df

RPI    INDPRO  UNRATE  UEMP5TO14  UEMP27OV  \
2003-01-01 00:00:00+00:00  12033.660   91.1369     5.8     2582.0    1760.0   
2003-01-02 00:00:00+00:00  12033.660   91.1369     5.8     2582.0    1760.0   
2003-01-03 00:00:00+00:00  12033.660   91.1369     5.8     2582.0    1760.0   
2003-01-06 00:00:00+00:00  12033.660   91.1369     5.8     2582.0    1760.0   
2003-01-07 00:00:00+00:00  12033.660   91.1369     5.8     2582.0    1760.0   
...                              ...       ...     ...        ...       ...   
2025-07-28 00:00:00+00:00  20625.729  103.8194     4.2     2034.0    1826.0   
2025-07-29 00:00:00+00:00  20625.729  103.8194     4.2     2034.0    1826.0   
2025-07-30 00:00:00+00:00  20625.729  103.8194     4.2     2034.0    1826.0   
2025-07-31 00:00:00+00:00  20625.729  103.8194     4.2     2034.0    1826.0   
2025-08-01 00:00:00+00:00  20646.475  103.9203     4.3     2049.0    1930.0   

                             PAYEMS   USGOOD  CES1021000001  USCONS   MANEMP  \
2003-01-01 00:00:00+00:00  130573.0  22146.0          504.4  6704.0  14867.0   
2003-01-02 00:00:00+00:00  130573.0  22146.0          504.4  6704.0  14867.0   
2003-01-03 00:00:00+00:00  130573.0  22146.0          504.4  6704.0  14867.0   
2003-01-06 00:00:00+00:00  130573.0  22146.0          504.4  6704.0  14867.0   
2003-01-07 00:00:00+00:00  130573.0  22146.0          504.4  6704.0  14867.0   
...                             ...      ...            ...     ...      ...   
2025-07-28 00:00:00+00:00  159518.0  21651.0          577.4  8302.0  12734.0   
2025-07-29 00:00:00+00:00  159518.0  21651.0          577.4  8302.0  12734.0   
2025-07-30 00:00:00+00:00  159518.0  21651.0          577.4  8302.0  12734.0   
2025-07-31 00:00:00+00:00  159518.0  21651.0          577.4  8302.0  12734.0   
2025-08-01 00:00:00+00:00  159540.0  21626.0          571.9  8295.0  12722.0   

                           ...  FEDFUNDS   WRESBAL   CURRCIR  WTREGEN  \
2003-01-01 00:00:00+00:00  ...      1.24     7.859   679.411    4.829   
2003-01-02 00:00:00+00:00  ...      1.24     7.859   679.411    4.829   
2003-01-03 00:00:00+00:00  ...      1.24     7.859   679.411    4.829   
2003-01-06 00:00:00+00:00  ...      1.24     7.859   679.411    4.829   
2003-01-07 00:00:00+00:00  ...      1.24     7.859   679.411    4.829   
...                        ...       ...       ...       ...      ...   
2025-07-28 00:00:00+00:00  ...      4.33  3362.277  2399.266  323.176   
2025-07-29 00:00:00+00:00  ...      4.33  3362.277  2399.266  323.176   
2025-07-30 00:00:00+00:00  ...      4.33  3347.414  2399.266  370.507   
2025-07-31 00:00:00+00:00  ...      4.33  3347.414  2399.266  370.507   
2025-08-01 00:00:00+00:00  ...      4.33  3347.414  2403.186  370.507   

                           BOGMBASE    WSHOSHO    WSHOTSL    WSHOBL  T5YIE  \
2003-01-01 00:00:00+00:00     699.3   629416.0   629406.0  226682.0   1.30   
2003-01-02 00:00:00+00:00     699.3   629416.0   629406.0  226682.0   1.30   
2003-01-03 00:00:00+00:00     699.3   629416.0   629406.0  226682.0   1.28   
2003-01-06 00:00:00+00:00     699.3   629416.0   629406.0  226682.0   1.31   
2003-01-07 00:00:00+00:00     699.3   629416.0   629406.0  226682.0   1.28   
...                             ...        ...        ...       ...    ...   
2025-07-28 00:00:00+00:00    5740.3  6343851.0  4206720.0  195493.0   2.47   
2025-07-29 00:00:00+00:00    5740.3  6343851.0  4206720.0  195493.0   2.48   
2025-07-30 00:00:00+00:00    5740.3  6329935.0  4206916.0  195493.0   2.48   
2025-07-31 00:00:00+00:00    5740.3  6329935.0  4206916.0  195493.0   2.46   
2025-08-01 00:00:00+00:00    5686.4  6329935.0  4206916.0  195493.0   2.38   

                           T10YIE  
2003-01-01 00:00:00+00:00    1.64  
2003-01-02 00:00:00+00:00    1.64  
2003-01-03 00:00:00+00:00    1.62  
2003-01-06 00:00:00+00:00    1.63  
2003-01-07 00:00:00+00:00    1.62  
...                           ...  
2025-07-28 00:00:00+00:00    2.44  
2025-07-29 00:00:00+0

In [28]:
# pd.set_option('display.max_rows', None)
first_valid_indices = df.apply(lambda col: col.first_valid_index())

print(first_valid_indices)

RPI         2003-01-01 00:00:00+00:00
INDPRO      2003-01-01 00:00:00+00:00
UNRATE      2003-01-01 00:00:00+00:00
UEMP5TO14   2003-01-01 00:00:00+00:00
UEMP27OV    2003-01-01 00:00:00+00:00
                       ...           
WSHOSHO     2003-01-01 00:00:00+00:00
WSHOTSL     2003-01-01 00:00:00+00:00
WSHOBL      2003-01-01 00:00:00+00:00
T5YIE       2003-01-01 00:00:00+00:00
T10YIE      2003-01-01 00:00:00+00:00
Length: 94, dtype: datetime64[ns, UTC]


### 2) Extracting complementary data

We get price data on gold spot price in USD through https://www.investing.com/currencies/xau-usd-historical-data.

In [34]:
data_path = os.path.join('data', 'US', 'xau_usd.csv')
data_path2 = os.path.join('data', 'US', 'xau_usd2.csv')
df_gold = pd.concat([pd.read_csv(data_path2, index_col=0, thousands=','), pd.read_csv(data_path, index_col=0, thousands=',')])

In [30]:
df_gold

Price     Open     High      Low Vol. Change %
Date                                                        
10/17/2025  4249.98  4328.75  4379.44  4186.91  NaN   -1.76%
10/16/2025  4326.18  4209.81  4330.91  4199.60  NaN    2.79%
10/15/2025  4208.58  4144.40  4218.42  4140.45  NaN    1.60%
10/14/2025  4142.31  4108.98  4180.39  4090.49  NaN    0.77%
10/13/2025  4110.70  4018.79  4117.14  4008.80  NaN    2.30%
...             ...      ...      ...      ...  ...      ...
01/07/2000   281.75   281.15   281.75   281.75  NaN    0.18%
01/06/2000   281.23   280.12   281.23   281.23  NaN    0.08%
01/05/2000   281.00   281.50   281.00   281.00  NaN   -0.51%
01/04/2000   282.45   288.75   282.45   282.45  NaN   -2.27%
01/03/2000   289.00   288.50   289.00   289.00  NaN    0.26%

[6715 rows x 6 columns]

In [31]:
df_gold.dtypes


Price       float64
Open        float64
High        float64
Low         float64
Vol.         object
Change %     object
dtype: object

In [35]:
df_gold.index = pd.to_datetime(df_gold.index.astype(str), utc=True)
df_gold = df_gold[["Price"]].rename(columns={"Price":'XAU_USD'})

We get index level data on the S&P500 through https://wrds-www.wharton.upenn.edu

In [36]:
data_path = os.path.join('data', 'US', 'sp500.csv')
df_sp500 = pd.read_csv(data_path, index_col=0)
df_sp500.index = pd.to_datetime(df_sp500.index, utc=True)

### 3) Merge data

We inner merge our three DataFrames on their date indices. 

In [37]:
df = df.merge(right=df_gold, how='inner', left_index=True, right_index=True).merge(right=df_sp500, how='inner', left_index=True, right_index=True)

In [39]:
data_path = os.path.join('data', 'US', 'us_data.csv')
df.to_csv(data_path, index_label='Date')